# Advanced Python
## `numpy` & `scipy` practice

In this notebook, you can see the solution of various non-trivial tasks with the use of modules `numpy` and `scipy`.

In [ ]:
import math
import time
import random
import warnings

import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt

from IPython.display import clear_output
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

SEED = 21
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

### Task №1

Implement the following operations for vectors using `np.einsum`:

1. ```np.sum(A)```
2. ```A * B```
3. ```np.inner(A, B)```
4. ```np.outer(A, B)```

In [ ]:
%%write_and_run einsum_task.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import random

SEED = 21
random.seed(SEED)
np.random.seed(SEED)

def task_00(A):
    res = np.einsum('i->', A)
    return res

def task_01(A, B):
    res = np.einsum('i,i->i', A, B)
    return res

def task_02(A, B):
    res = np.einsum('i,i->', A, B)
    return res

def task_03(A, B):
    res = np.einsum('i,j->ij', A, B)
    return res


### Task №2

Gaussian filter is a powerful tool in image processing, which allows you to get rid of unnecessary noise in the picture (and also gives you the opportunity to blur the picture)

To illustrate how the filter works:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/62/Cappadocia_Gaussian_Blur.svg/800px-Cappadocia_Gaussian_Blur.svg.png)

You need to implement a two-dimensional Gaussian filter using numpy. How it is calculated:

$$G = e^{- \frac{(D - μ)^{2}}{2 \sigma^{2}} }$$

where

$$D = \sqrt{X^2 + Y^2} $$

$\mu$ and $\sigma$ are constants (mean and standard deviation)

Calculate the Gaussian filter on the coordinates from -1 to 1 (10 points on each axis, hint: use `np.meshgrid`)

In [ ]:
%%write_and_run gauss_filter.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np

def gauss_filter(sigma = 1.0, mu = 0.0):
    x = np.linspace(-1, 1, 10, endpoint=True)
    y = np.linspace(-1, 1, 10, endpoint=True)
    xv, yv = np.meshgrid(x, y)

    nom = (np.sqrt(np.power(xv, 2) + np.power(yv, 2)) - mu) ** 2
    denom = 2 * (sigma ** 2)

    res = np.exp(- nom / denom)
    return res


### Task №3

Need to find top-k largest elements in an array using `numpy`

In [ ]:
%%write_and_run greatest_task.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import random

SEED = 21
random.seed(SEED)
np.random.seed(SEED)

def find_largest_element(array, n=7):
    # apply inplace to avoid creating a new object
    array.sort()
    # take top-n largest elements
    return array[-n:]


### Task №4

**Game of life**

The place of the game action is a plane marked into cells, which can be infinite, limited or closed. Each cell on this surface has eight neighbors surrounding it and can be in two states: alive (filled) or dead (empty). The distribution of living cells at the beginning of the game is called the first generation. Each next generation is calculated based on the previous one according to these rules:
* in an empty (dead) cell with three living cells neighboring it, life originates
* if a living cell has two or three living neighbors and the cell is not on the border, this cell continues to live; otherwise (if there are less than two or more than three living neighbors) the cell dies («from loneliness» or «from overpopulation»)

The game stops if there is 
* there are no «alive» cell left on the field
* the configuration on the next step will exactly (without shifts and turns) repeat itself on one of the earlier steps (a periodic configuration is formed)
* at the next step none of the cells changes its state (a special case of the previous rule, a stable configuration is formed)

Your task is to implement the game using `numpy`

In [ ]:
%%write_and_run game_of_life.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import random
from scipy.signal import convolve2d

SEED=21
random.seed(SEED)
np.random.seed(SEED)

def game_of_life_next_step(array):
    """
    Function documentation from scipy:
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html
    """
    # Idea: walk across the game field with a convolution kernel symbolizing a pattern of possible neighbors
    # then the result will be the number of neighbors
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])
    # mode='same' -- to make the output the same size as the array
    # boundary='wrap' -- symbolizes the looping of the field
    neighbors_counts = convolve2d(array, kernel, mode='same', boundary='wrap')

    # end-of-life conditions
    array[(array == 1) & (neighbors_counts < 2)] = 0
    array[(array == 1) & (neighbors_counts > 3)] = 0
    # conditions for the emergence of life
    array[(array == 0) & (neighbors_counts == 3)] = 1

    # the cells on the border must die
    # 1-st row
    array[0, :] = 0
    # 1-st col
    array[:, 0] = 0
    # last row
    array[-1, :] = 0
    # last col
    array[:, -1] = 0

    return array


### Task №5

Recall the **differential evolution algorithm** we went over in the seminar:

1. Population Initialization: Start by creating a random population where each individual (vector) represents a set of parameters to be optimized. These parameters can be represented by real numbers.
2. Mutation Operator: For each individual in the population, a mutant individual is created. This is done by combining parameters from several randomly selected individuals using the mutation operator. Typically, the mutation operator might look like this:
$mutant = a + mutcoeff * (b - c)$
3. Crossover (Recombination) Operator: The mutant individual is combined with the original individual using the crossover (recombination) operator. This operator helps to determine which parameters to keep from the original individual and which parameters to replace with mutant values.
4. Progeny Evaluation: Evaluate the quality of the progeny obtained after crossover using the target function.
5. Selection: This is where the selection of an individual for the next generation takes place. Compare the quality of the offspring and the original individual. If the descendant turns out to be better than the original individual, it becomes part of the next generation. Otherwise, we keep the original individual.
6. Repeat steps 3-6: Repeat the operations of mutation, crossover, evaluation, and selection of individuals for the next generation for a certain number of iterations or until a specified stopping criterion is reached (e.g., maximum number of iterations or achievement of a specified accuracy).

**The task:**
1. Implement population socialization using different distributions such as:

    a. [LatinHypercube](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.LatinHypercube.html)

    b. [Halton](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.Halton.html)

    c. [Sobol](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.Sobol.html)

2. Implement various mutation operators:
    a. *Rand2* - selection of 5 random elements $mutant = a + mutcoeff * (b - c) + mutcoeff * (d - e)$
    b. *Best1* - use of the best element, excluding the current best and current iteration element in the mutation $mutant = best + mutcoeff * (b - c)$
    c. *Rand-to-p-best* - using one of the $p$ percent best elements, excluding the current best and current iteration element in the mutation $mutant$ = *random better element* $+ mutcoeff * (b - c)$
3. Realize different types of selection

    Now the selection is performed on the descendant and the original invididid. In this task, we need to implement the comparison not on the source indvididid, but on:
    
    a. By the worst individual in the indvididid (ancestor) population
    
    b. Randomly select an individual from the worst indvididid relative to the original indvididid. If there are none, choose the original one.
    
    c. Select a random indwid not including the best and current indwid


**Important**:
1. Use `SEED` everywhere, including when initializing initial distributions.
2. Use ``np.random.choice(replace=False)`` instead of ``random.choice``.
3. Do not delete the line ```p = np.random.uniform(p_min, p_max)```.
4. After initialization, remember to mix: ```population.random(n=population_size)```.
5. Try not to use loops.
6. For Sobol initialization, try using ```random_base2``` and ```np.log2```, if not satisfactory, then as usual.
7. Try using ```np.setdiff1d(..., assume_unique=True)```
8. Instead of ```min```/```max```, use ```np.min```, ```np.max```.
9. The mutation strategy ```random_among_worst``` does not require sorting

In [ ]:
%%write_and_run diff_evolution.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import random
from scipy.stats import qmc
import math


def differential_evolution(fobj, bounds, mutation_coefficient=0.5,
                           crossover_coefficient=0.5, population_size=50, iterations=50,
                           init_setting='random', mutation_setting='rand1',
                           selection_setting='current', p_min=0.1, p_max=0.2):
    # Initializing the population and obtaining primary results
    SEED = 7
    random.seed(SEED)
    np.random.seed(SEED)
    bounds = np.array(bounds)
    dimensions = len(bounds)
    # Random initialization
    if init_setting == 'LatinHypercube':
        population = qmc.LatinHypercube(dimensions, seed=SEED)
        assert population.__class__ == qmc.LatinHypercube
        population = population.random(n=population_size)
    elif init_setting == 'Halton':
        population = qmc.Halton(dimensions, seed=SEED)
        assert population.__class__ == qmc.Halton
        population = population.random(n=population_size)
    elif init_setting == 'Sobol':
        population = qmc.Sobol(dimensions, seed=SEED)
        assert population.__class__ == qmc.Sobol
        population = population.random(n=population_size)
    else:
        population = np.random.rand(population_size, dimensions)
    min_bound, max_bound = bounds.T
    diff = np.fabs(min_bound - max_bound)
    population_denorm = min_bound + population * diff
    fitness = np.asarray([fobj(ind) for ind in population_denorm])
    # Find the best index
    best_idx = np.argmin(fitness)
    best = population_denorm[best_idx]
    for iteration in range(iterations):
        for population_index in range(population_size):
            idxs = np.setdiff1d(np.arange(population_size), [best_idx, population_index], assume_unique=True)
            # Selection of three random elements
            # Mutation operator
            if mutation_setting == 'rand2':
                a, b, c, d, e = population[np.random.choice(idxs, 5, replace=False)]
                mutant = np.clip(a + mutation_coefficient * (b - c) + mutation_coefficient * (d - e), 0, 1)

                assert 'e' in locals(), "This assert checks that you have written the formula accurately"
                assert 'd' in locals(), "This assert checks that you have written the formula accurately"
            elif mutation_setting == 'best1':
                # sort by target function
                argsort = np.argsort(fitness)
                # remove the best object and the current one
                argsort_no_best_and_cur = argsort[np.isin(argsort, [best_idx, population_index], invert=True)]
                # take the best of the rest
                index_of_best1 = argsort_no_best_and_cur[0]
                # remove the best of the remaining ones from the selection list
                idxs = np.delete(idxs, np.where(idxs == index_of_best1)[0])
                # select two more objects
                b, c = population[np.random.choice(idxs, 2, replace=False)]

                assert index_of_best1 not in idxs,  "This assertion verifies that you will not use the chosen one object to select b and c so that you do not repeat yourself"
                assert index_of_best1 != population_index, "This assert verifies that you have not taken the index of the current individual"
                assert index_of_best1 != best_idx, "This assert verifies that you have not taken the index of the best individual"
                if iteration == 0:
                    for idx in idxs: assert np.array_equal(population[index_of_best1], population[idx]) is False, "This assert verifies that the selected index is correct"
                    assert np.array_equal(population[index_of_best1], population[population_index]) is False, "This assert verifies that the selected index is correct"
                    assert np.array_equal(population[index_of_best1], population[best_idx]) is False, "This assert verifies that the selected index is correct"

                mutant = np.clip(population[index_of_best1] + mutation_coefficient * (b - c), 0, 1)
            elif mutation_setting == 'rand_to_p_best1':
                p = np.random.uniform(p_min, p_max)  # don't delete
                # sort by target function
                argsort = np.argsort(fitness)
                # remove the best object and the current one
                argsort_no_best_and_cur = argsort[np.isin(argsort, [best_idx, population_index], invert=True)]
                # take a quantile
                to_choose = argsort_no_best_and_cur[:int(len(argsort_no_best_and_cur) * p)]
                # select one object from those in the quantile
                index_of_rand_to_p_best1 = np.random.choice(to_choose, 1, replace=False)[0]
                # delete the taken object
                idxs = np.delete(idxs, np.where(idxs == index_of_rand_to_p_best1)[0])
                # select two more objects
                b, c = population[np.random.choice(idxs, 2, replace=False)]

                assert 'a' not in locals()
                assert index_of_rand_to_p_best1 not in idxs, "This assertion verifies that you will not use the chosen one object to select b and c so that you do not repeat yourself"
                assert index_of_rand_to_p_best1 != population_index, "This assert verifies that you have not taken the index of the current individual"
                assert index_of_rand_to_p_best1 != best_idx, "This assert verifies that you have not taken the index of the best individual"
                if iteration == 0:
                    for idx in idxs: assert np.array_equal(population[index_of_rand_to_p_best1], population[idx]) is False, "This assert verifies that the selected index is correct"
                    assert np.array_equal(population[index_of_rand_to_p_best1], population[population_index]) is False, "This assert verifies that the selected index is correct"
                    assert np.array_equal(population[index_of_rand_to_p_best1], population[best_idx]) is False, "This assert verifies that the selected index is correct"

                mutant = np.clip(population[index_of_rand_to_p_best1] + mutation_coefficient * (b - c), 0, 1)
            else:
                a, b, c = population[np.random.choice(idxs, 3, replace = False)]
                mutant = np.clip(a + mutation_coefficient * (b - c), 0, 1)
            # Crossover operator
            cross_points = np.random.rand(dimensions) < crossover_coefficient
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            # Recombination (replacement with mutant values)
            trial = np.where(cross_points, mutant, population[population_index])
            trial_denorm = min_bound + trial * diff
            # Offspring's valuation
            result_of_evolution = fobj(trial_denorm)
            # Breeding
            if selection_setting == 'worst':
                selection_index = np.argmax(fitness)
            elif selection_setting == 'random_among_worst':
                # make a mask of objects worse than the current one
                bad_mask = fitness > result_of_evolution
                if np.sum(bad_mask) > 0:
                    # take the ones that are worse and pick one out of them 
                    bad_guys = np.nonzero(bad_mask)[0]
                    selection_index = np.random.choice(bad_guys, 1, replace=False)
                else:
                    # If there aren't any, we take this object 
                    selection_index = population_index
            elif selection_setting == 'random_selection':
                selection_index = np.random.choice(idxs, 1)
            else:
                selection_index = population_index
            if result_of_evolution < fitness[selection_index]:
                fitness[selection_index] = result_of_evolution
                population[selection_index] = trial
                if result_of_evolution < fitness[best_idx]:
                    best_idx = selection_index
                    best = trial_denorm
        yield best, fitness[best_idx]
